In [1]:
!wget http://www.eraserbenchmark.com/zipped/multirc.tar.gz
!tar -xvzf multirc.tar.gz
!mv multirc/val.jsonl multirc/dev.jsonl

--2022-09-05 16:27:49--  http://www.eraserbenchmark.com/zipped/multirc.tar.gz
Resolving www.eraserbenchmark.com (www.eraserbenchmark.com)... 54.161.241.46, 54.237.133.81, 18.205.222.128, ...
Connecting to www.eraserbenchmark.com (www.eraserbenchmark.com)|54.161.241.46|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1667550 (1.6M) [application/gzip]
Saving to: ‘multirc.tar.gz’

multirc.tar.gz      100%[===================>]   1.59M  3.02MB/s    in 0.5s    

2022-09-05 16:27:50 (3.02 MB/s) - ‘multirc.tar.gz’ saved [1667550/1667550]

multirc/
multirc/docs/
multirc/docs/News_CNN_cnn-3b31c717af0ec349ef0a067373d6444e1a05c414.txt
multirc/docs/Society_Law_and_Justice_masc-A_defense_of_Michael_Moore-10.txt
multirc/docs/Science-textbook_science-g3-10.txt
multirc/docs/wikiMovieSummaries_3965765.txt
multirc/docs/Science-textbook_science-g4-47.txt
multirc/docs/Sept11-reports_oanc-chapter-6-22.txt
multirc/docs/Sept11-reports_oanc-chapter-1-6.txt
multirc/docs/Fiction-stories_

In [1]:
import json
from typing import Dict, List, Set, Tuple, Union, FrozenSet
import os
from itertools import chain

def load_documents(data_dir: str, docids: Set[str] = None) -> Dict[str, List[List[str]]]:
    """Loads a subset of available documents from disk.

    Each document is assumed to be serialized as newline ('\n') separated sentences.
    Each sentence is assumed to be space (' ') joined tokens.
    """
    if os.path.exists(os.path.join(data_dir, 'docs.jsonl')):
        assert not os.path.exists(os.path.join(data_dir, 'docs'))
        return load_documents_from_file(data_dir, docids)

    docs_dir = os.path.join(data_dir, 'docs')
    res = dict()
    if docids is None:
        docids = sorted(os.listdir(docs_dir))
    else:
        docids = sorted(set(str(d) for d in docids))
    for d in docids:
        with open(os.path.join(docs_dir, d), 'r') as inf:
            lines = [l.strip() for l in inf.readlines()]
            lines = list(filter(lambda x: bool(len(x)), lines))
            tokenized = [list(filter(lambda x: bool(len(x)), line.strip().split(' '))) for line in lines]
            res[d] = tokenized
    return res
def load_flattened_documents(data_dir: str, docids: Set[str]) -> Dict[str, List[str]]:
    """Loads a subset of available documents from disk.

    Returns a tokenized version of the document.
    """
    unflattened_docs = load_documents(data_dir, docids)
    flattened_docs = dict()
    for doc, unflattened in unflattened_docs.items():
        flattened_docs[doc] = list(chain.from_iterable(unflattened))
    return flattened_docs

documents = load_flattened_documents('multirc', docids=None)
def read_annotations(json_file) :
    anns = [json.loads(line) for line in open(json_file)]
    for a in anns :
        doc_id = a['annotation_id'].split(':')
        assert len(doc_id) == 3
        doc_id = doc_id[0]
        a['document'] = " ".join(documents[doc_id])
        a['label'] = a['classification']
        del a['classification']
        del a['docids']
        a['rationale'] = []
        for evgroup in a['evidences'] :
            for ev in evgroup :
                assert ev['docid'] == doc_id
                a['rationale'].append((ev['start_token'], ev['end_token']))
        del a['evidences']
        del a['query_type']
        
    return anns

In [2]:
import os
os.makedirs('data/', exist_ok=True)

In [3]:
for key in ['train', 'dev', 'test'] :
    ann = read_annotations('multirc/' + key + '.jsonl')
    with open('data/' + key + '.jsonl', 'w') as f :
        f.write('\n'.join([json.dumps(line) for line in ann]))

In [5]:
for key in ['train', 'dev', 'test'] :
    ann = read_annotations('multirc/' + key + '.jsonl')
    with open('data/' + key + '.csv', 'w') as f :
        f.write('\n'.join([json.dumps(line) for line in ann]))

In [9]:
import pandas as pd    
jsonObj = pd.read_json(path_or_buf='multirc/dev.jsonl', lines=True)
print(jsonObj['evidences'].head())

                                     annotation_id classification  docids  \
0  Fiction-stories-masc-The_Black_Willow-0.txt:0:0          False     NaN   
1  Fiction-stories-masc-The_Black_Willow-0.txt:0:1          False     NaN   
2  Fiction-stories-masc-The_Black_Willow-0.txt:0:2           True     NaN   
3  Fiction-stories-masc-The_Black_Willow-0.txt:0:3           True     NaN   
4  Fiction-stories-masc-The_Black_Willow-0.txt:0:4           True     NaN   

                                           evidences  \
0  [[{'docid': 'Fiction-stories-masc-The_Black_Wi...   
1  [[{'docid': 'Fiction-stories-masc-The_Black_Wi...   
2  [[{'docid': 'Fiction-stories-masc-The_Black_Wi...   
3  [[{'docid': 'Fiction-stories-masc-The_Black_Wi...   
4  [[{'docid': 'Fiction-stories-masc-The_Black_Wi...   

                                               query  query_type  
0  Name few objects said to be in or on Allan 's ...         NaN  
1  Name few objects said to be in or on Allan 's ...         NaN  

In [11]:
print(jsonObj['evidences'][1])

[[{'docid': 'Fiction-stories-masc-The_Black_Willow-0.txt', 'end_sentence': 1, 'end_token': 13, 'start_sentence': 0, 'start_token': 0, 'text': 'Allan sat down at his desk and pulled the chair in close .'}, {'docid': 'Fiction-stories-masc-The_Black_Willow-0.txt', 'end_sentence': 2, 'end_token': 29, 'start_sentence': 1, 'start_token': 13, 'text': 'Opening a side drawer , he took out a piece of paper and his inkpot .'}, {'docid': 'Fiction-stories-masc-The_Black_Willow-0.txt', 'end_sentence': 3, 'end_token': 57, 'start_sentence': 2, 'start_token': 29, 'text': "After filling his pen , Allan looked at his paper in the orange glow from the lantern set back in the desk 's right - hand corner ."}]]


In [7]:
with open('multirc/dev.jsonl', 'r') as json_file:
    json_list = list(json_file)

for json_str in json_list:
    result = json.loads(json_str)
    print(f"result: {result}")
    print(isinstance(result, dict))

result: {'annotation_id': 'Fiction-stories-masc-The_Black_Willow-0.txt:0:0', 'classification': 'False', 'docids': None, 'evidences': [[{'docid': 'Fiction-stories-masc-The_Black_Willow-0.txt', 'end_sentence': 1, 'end_token': 13, 'start_sentence': 0, 'start_token': 0, 'text': 'Allan sat down at his desk and pulled the chair in close .'}, {'docid': 'Fiction-stories-masc-The_Black_Willow-0.txt', 'end_sentence': 2, 'end_token': 29, 'start_sentence': 1, 'start_token': 13, 'text': 'Opening a side drawer , he took out a piece of paper and his inkpot .'}, {'docid': 'Fiction-stories-masc-The_Black_Willow-0.txt', 'end_sentence': 3, 'end_token': 57, 'start_sentence': 2, 'start_token': 29, 'text': "After filling his pen , Allan looked at his paper in the orange glow from the lantern set back in the desk 's right - hand corner ."}]], 'query': "Name few objects said to be in or on Allan 's desk || Eraser", 'query_type': None}
True
result: {'annotation_id': 'Fiction-stories-masc-The_Black_Willow-0.txt